In [ ]:
%matplotlib inline
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import numpy as np
import matplotlib.pyplot as plt
import argparse

from torchvision import datasets
import torchvision.transforms as transforms

from torchvision.utils import save_image

import torch
import torch.nn as nn
import torch.nn.functional as F



In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
architecture = argparse.ArgumentParser()

architecture.add_argument("--img_dem", default=28, type=int, help="dimensions of image")
architecture.add_argument("--batch_size", default=64, type=int, help="size of each batch for training")
architecture.add_argument("--latent_space", default=100, type=int, help="dim of starting point of generator")
architecture.add_argument("--num_classes", default=10, type=int, help="number of classes")
architecture.add_argument("--lr", default=0.001, type=float, help="Learning rate")
architecture.add_argument("--epochs", default=200, type=int, help="no of iterations over dataset")
architecture.add_argument('-f')

features = architecture.parse_args()
print(features)

Namespace(batch_size=64, epochs=200, f='/root/.local/share/jupyter/runtime/kernel-3372a331-df7f-45ba-9341-f74cca6f73a8.json', img_dem=28, latent_space=100, lr=0.001, num_classes=10)


In [ ]:
os.makedirs('mnist', exist_ok=True)
train_data = torch.utils.data.DataLoader(datasets.MNIST("/content/drive/MyDrive/mnist", 
                                            train=True, 
                                            download=True, 
                                            transform=transforms.Compose(
                                                    [transforms.Resize(features.img_dem), 
                                                    transforms.ToTensor(), 
                                                    transforms.Normalize([0.5], [0.5])]
                                                )
                                              ), 
                                            batch_size= features.batch_size,
                                            shuffle=True
                                        )

In [ ]:
def net(inp, out, dropout=False, norm=False, sigmoid=False):
      layers = []
      layers.append(nn.Linear(inp, out))

      if dropout:
        layers.append(nn.Dropout(0.4))

      if norm:
        layers.append(nn.BatchNorm1d(out, 0.5))

      layers.append(nn.LeakyReLU(0.2, inplace=True))

      if sigmoid:
        nn.Sigmoid()

      return layers

In [ ]:
class Generator(nn.Module):
  def __init__(self):
      super(Generator, self).__init__()

      self.embed_label = nn.Embedding(features.num_classes, features.num_classes)

      self.model = nn.Sequential(
          *net(features.latent_space + features.num_classes, 128),
          *net(128, 256, False, True),
          *net(256, 512, False, True),
          *net(512, features.img_dem * features.img_dem, False, True, True)
      )

  def forward(self, labels, inp_noice):
    emb = self.embed_label(labels)
    input_conc = torch.cat((emb, inp_noice), -1)
    out = self.model(input_conc)
    out = out.view(out.size(0), *(1,28,28))
    return out

In [ ]:
class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator, self).__init__()

    self.embed_label = nn.Embedding(features.num_classes, features.num_classes)

    self.model = nn.Sequential(
        *net(784 + features.num_classes, 512),
        *net(512, 256, True),
        *net(256, 128, True),
        *net(128, 64, True),
        *net(64, 1)
    )


  def forward(self, labels, image):
    emb = self.embed_label(labels)
    input_conc = torch.cat((image.view(image.size(0), -1), emb, -1))
    img = self.model(input_conc)
    return img

In [ ]:
G = Generator()
D = Discriminator()

loss = torch.nn.MSELoss()

In [ ]:
cuda = True if torch.cuda.is_available() else False

if cuda:
  G.cuda()
  D.cuda()
  loss.cuda()

In [ ]:
optimizer_G = torch.optim.Adam(G.parameters(), lr= features.lr)
optimizer_D = torch.optim.Adam(D.parameters(), lr= features.lr)

In [ ]:
os.makedirs("/content/drive/MyDrive/images", exist_ok=True)

def save_img(rows, batches_done):
  labels = np.array([num for _ in range(rows) for num in range(rows)])
  labels = labels.ToTensor.cuda()

  z = np.random.normal(0, 1, (rows ** 2, features.latent_space))
  z = z.ToTensor.cuda()

  generated_img = G(labels, z)
  save_image(generated_img.data, "/content/drive/MyDrive/images/%d.png" % batches_done, nrow=rows, normalize=True)
  

In [ ]:
for epoch in range(features.epochs):
  for i, (imgs, labels) in enumerate(train_data):

    batch_size = imgs.shape[0]
    print(batch_size)

    valid = Variable(FloatTensor(batch_size, 1).fill_(1.0), requires_grad=False)
    fake = Variable(FloatTensor(batch_size, 1).fill_(0.0), requires_grad=False)

    imgs = Variable(imgs.type(FloatTensor))
    labels = Variable(labels.type(LongTensor))

    '''
    ##########################
        GENERATOR TRAINING
    ##########################
    '''

    optimizer_G.zero_grad()

    noice = Variable(FloatTensor(np.random.normal(0, 1, (batch_size, features.latent_space))))
    gen_labels = Variable(LongTensor(np.random.randint(0, features.num_classes, batch_size)))

    generated_img = G(gen_labels, noice)

    gen_out = D(gen_labels, generated_img)
    gen_loss = loss(gen_out, valid)

    gen_loss.backward()
    optimizer_G.step()

    '''
    ##########################
      DISCRIMINATOR TRAINING
    ##########################
    '''

    optimizer_D.zero_grad()

    dis_out_real = D(labels, imgs)
    dis_real_loss = loss(dis_out_real, valid)

    dis_out_fake = D(gen_labels, generated_img.detach())
    dis_fake_loss = loss(dis_out_fake, fake)

    dis_loss = (dis_real_loss + dis_fake_loss) / 2

    dis_loss.backward()
    optimizer_D.step()

    print(
      "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
      % (epoch, features.epochs, i, len(train_data), dis_loss.item(), gen_loss.item())
    )

    batches_done = epoch * len(train_data) + i
    if batches_done % 400 == 0:
        save_img(rows=10, batches_done=batches_done)

Streaming output truncated to the last 5000 lines.
64
[Epoch 197/200] [Batch 314/938] [D loss: 0.060286] [G loss: 0.735257]
64
[Epoch 197/200] [Batch 315/938] [D loss: 0.071807] [G loss: 0.677675]
64
[Epoch 197/200] [Batch 316/938] [D loss: 0.045302] [G loss: 0.840071]
64
[Epoch 197/200] [Batch 317/938] [D loss: 0.070189] [G loss: 0.909619]
64
[Epoch 197/200] [Batch 318/938] [D loss: 0.090280] [G loss: 1.019105]
64
[Epoch 197/200] [Batch 319/938] [D loss: 0.041791] [G loss: 0.910172]
64
[Epoch 197/200] [Batch 320/938] [D loss: 0.032126] [G loss: 0.822458]
64
[Epoch 197/200] [Batch 321/938] [D loss: 0.086073] [G loss: 0.585013]
64
[Epoch 197/200] [Batch 322/938] [D loss: 0.029002] [G loss: 0.828343]
64
[Epoch 197/200] [Batch 323/938] [D loss: 0.046833] [G loss: 0.834275]
64
[Epoch 197/200] [Batch 324/938] [D loss: 0.050721] [G loss: 0.944487]
64
[Epoch 197/200] [Batch 325/938] [D loss: 0.044880] [G loss: 0.844982]
64
[Epoch 197/200] [Batch 326/938] [D loss: 0.027407] [G loss: 0.820411]


In [ ]:
G.load_state_dict(torch.load("/content/drive/MyDrive/GANsmnistmodel/g.pt"))
G = G.to("cuda")
G.eval()

Generator(
  (embed_label): Embedding(10, 10)
  (model): Sequential(
    (0): Linear(in_features=110, out_features=128, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Linear(in_features=128, out_features=256, bias=True)
    (3): BatchNorm1d(256, eps=0.5, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Linear(in_features=256, out_features=512, bias=True)
    (6): BatchNorm1d(512, eps=0.5, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Linear(in_features=512, out_features=784, bias=True)
    (9): BatchNorm1d(784, eps=0.5, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
  )
)

In [ ]:
labels = np.random.randint(3, 8, 1)
labels = torch.LongTensor(labels).cuda()


z = np.random.normal(0, 1, (1, 100))
z = torch.FloatTensor(z).cuda()
print(z.shape)

generated_img = G(labels, z)

save_image(generated_img.data, "/content/drive/MyDrive/images/0000.png", nrow=1, normalize=True)

RuntimeError: ignored